# Lab 1

# Team Members:
# Achchala Deepan - 20943939
# Manjary Muruganandan - 20950662

In [13]:
from cplex import Cplex
from docplex.mp.model import Model

### Parameters

In [14]:
# Facility IDs and attributes
facilities = ['F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7']
setup_cost = {'F1': 1000, 'F2': 1200, 'F3': 1100, 'F4': 1300, 'F5': 1400, 'F6': 1250, 'F7': 1350}
capacity = {'F1': 50, 'F2': 60, 'F3': 40, 'F4': 70, 'F5': 55, 'F6': 65, 'F7': 45}

# Customer IDs and demand
customers = ['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10']
demand = {'C1': 15, 'C2': 10, 'C3': 20, 'C4': 25, 'C5': 30, 'C6': 10, 'C7': 15, 'C8': 10, 'C9': 25, 'C10': 20}

# Transportation info
rate = 3.5  # per unit per km
revenue_per_unit = 1000  # per unit demand fulfilled

# Distance between each facility and customer (example only — replace with real data)
distance = {
    ('F1','C1'):4, ('F1','C2'):6, ('F1','C3'):9, ('F1','C4'):5, ('F1','C5'):8, ('F1','C6'):7, ('F1','C7'):6, ('F1','C8'):5, ('F1','C9'):4, ('F1','C10'):3,
    ('F2','C1'):5, ('F2','C2'):4, ('F2','C3'):7, ('F2','C4'):6, ('F2','C5'):5, ('F2','C6'):4, ('F2','C7'):7, ('F2','C8'):8, ('F2','C9'):6, ('F2','C10'):5,
    ('F3','C1'):6, ('F3','C2'):3, ('F3','C3'):4, ('F3','C4'):7, ('F3','C5'):8, ('F3','C6'):6, ('F3','C7'):5, ('F3','C8'):4, ('F3','C9'):6, ('F3','C10'):4,
    ('F4','C1'):3, ('F4','C2'):5, ('F4','C3'):6, ('F4','C4'):4, ('F4','C5'):7, ('F4','C6'):6, ('F4','C7'):4, ('F4','C8'):5, ('F4','C9'):7, ('F4','C10'):5,
    ('F5','C1'):5, ('F5','C2'):4, ('F5','C3'):6, ('F5','C4'):5, ('F5','C5'):6, ('F5','C6'):4, ('F5','C7'):5, ('F5','C8'):6, ('F5','C9'):5, ('F5','C10'):5,
    ('F6','C1'):4, ('F6','C2'):3, ('F6','C3'):5, ('F6','C4'):4, ('F6','C5'):7, ('F6','C6'):6, ('F6','C7'):5, ('F6','C8'):6, ('F6','C9'):6, ('F6','C10'):4,
    ('F7','C1'):6, ('F7','C2'):6, ('F7','C3'):7, ('F7','C4'):6, ('F7','C5'):5, ('F7','C6'):5, ('F7','C7'):6, ('F7','C8'):5, ('F7','C9'):4, ('F7','C10'):5,
}

### Model

In [15]:
mdl = Model(name="CFLP_ProfitMax")

### Decision Variables

In [16]:
# Whether a facility is opened
x = mdl.binary_var_dict(facilities, name='Open')

# Whether a customer is assigned to a facility
y = mdl.binary_var_dict([(f, c) for f in facilities for c in customers], name='Assign')

# Amount shipped from facility to customer
q = mdl.continuous_var_dict([(f, c) for f in facilities for c in customers], name='Ship')


### Constraints

In [18]:
# 1. Each customer assigned to at most 1 facility
for c in customers:
    mdl.add_constraint(mdl.sum(y[f, c] for f in facilities) <= 1, f"AssignLimit_{c}")

# 2. Ship only if customer assigned
for f in facilities:
    for c in customers:
        mdl.add_constraint(q[f, c] <= demand[c] * y[f, c], f"ShipIfAssigned_{f}_{c}")

# 3. Capacity constraint for each facility
for f in facilities:
    mdl.add_constraint(mdl.sum(q[f, c] for c in customers) <= capacity[f] * x[f], f"CapacityLimit_{f}")

# 4. Max 3 facilities can be opened
mdl.add_constraint(mdl.sum(x[f] for f in facilities) <= 3, "Max3Facilities")

docplex.mp.LinearConstraint[Max3Facilities](Open_F1+Open_F2+Open_F3+Open_F4+Open_F5+Open_F6+Open_F7,LE,3)

### Objective Function

In [19]:
revenue = mdl.sum(q[f, c] * revenue_per_unit for f in facilities for c in customers)
transport_cost = mdl.sum(q[f, c] * rate * distance[f, c] for f in facilities for c in customers)
setup_total = mdl.sum(setup_cost[f] * x[f] for f in facilities)

mdl.maximize(revenue - transport_cost - setup_total)

### Solve

In [20]:
solution = mdl.solve(log_output=True)

Version identifier: 22.1.1.0 | 2022-11-28 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
Found incumbent of value 0.000000 after 0.00 sec. (0.01 ticks)
Tried aggregator 2 times.
MIP Presolve eliminated 87 rows and 0 columns.
Aggregator did 7 substitutions.
Reduced MIP has 81 rows, 140 columns, and 280 nonzeros.
Reduced MIP has 70 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.43 ticks)
Probing time = 0.00 sec. (0.07 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 81 rows, 140 columns, and 280 nonzeros.
Reduced MIP has 70 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.21 ticks)
Probing time = 0.00 sec. (0.07 ticks)
Clique table members: 10.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 8 threads.
Root relaxation solution time = 0.00 sec. (0.33 ticks)

        Nodes                                         Cuts

### Output

In [21]:
if solution:
    print("Total Profit:", mdl.objective_value)
    print("Facilities Opened:")
    for f in facilities:
        if x[f].solution_value > 0.5:
            print(f"  {f}")
    print("\nCustomer Assignments and Shipping:")
    for f in facilities:
        for c in customers:
            if y[f, c].solution_value > 0.5:
                print(f"  {c} assigned to {f}, Units: {q[f, c].solution_value}")
else:
    print("No feasible solution found.")

Total Profit: 173597.5
Facilities Opened:
  F1
  F4
  F6

Customer Assignments and Shipping:
  C9 assigned to F1, Units: 25.0
  C10 assigned to F1, Units: 20.0
  C1 assigned to F4, Units: 15.0
  C5 assigned to F4, Units: 30.0
  C7 assigned to F4, Units: 15.0
  C8 assigned to F4, Units: 10.0
  C2 assigned to F6, Units: 10.0
  C3 assigned to F6, Units: 20.0
  C4 assigned to F6, Units: 25.0
  C6 assigned to F6, Units: 10.0
